In [1]:
# Generelle moduler og funksjonsbeskrivelser brukt i forelesningen
from numpy import sin, cos, pi, exp
import numpy as np
import scipy.signal as sig
import matplotlib.pyplot as plt

%matplotlib ipympl

<img src="NTNU_Logo.png" align="left" style="width: 30%">
<br clear="all" />
<br></br>

# Signalbehandling og Digitale Kontrollere

* **Emne IELEA2302 - Signalbehandling**
* **Uke 16, 2021**
* **Relevant Pensummateriell:**
    * *Filterstrukturer: delkapittel 10-4 i boka*
    * *Avbildning av transferfunksjoner inngår ikke i pensum*
* **Underviser: Kai Erik Hoff**

# Tema:
* Diskretisering av tidskontinuerlige modeller
    * Kontrollteori som anvendelsesområde for Digitale Filtre
    * Poler og nullpunkt i s-planet og z-planet
    * Impulsinvarians - en metode for systemavbildning i z-planet
* Implementasjon av filtermodeller
    * Direkte Form 1
    * Direkte Form 2
    * Transponert Direkte Form
* Oppsummering digitale filtre

# Motivasjon

* Implementasjon av en regulator vil svært ofte bruke en datamaskin som opererer med diskréte sampleverdier.
* Vi trenger å finne et *Digitalt Filter* som kan realisere en ønsket regulator med f.eks. transferfunksjon $G_c(s)$.
<img src="Figurer/16_DigitalKontroller/Fig1_sloyfe.png" style="width: 60%; margin-left: 200px" />

* Figuren har opphav fra et annet emne, og bruker noe annen notasjon. Fra og med neste slide bruker vi "vanlig" signalbehandlingsnotasjon.

# Poler i S-planet
* Én pol $s=p$ i s-planet representerer et førseordens filter / system.
    * Transferfunksjon: $H(s) = \frac{1}{s-p}$
    * Impulsrespons: $h(t) = e^{p\cdot t} \cdot u(t)$
<img src="Figurer/16_DigitalKontroller/Fig2_Spole.png" style="width: 80%" />

# Poler i Z-planet
* Én pol $z=\hat{p}$ i Z-planet representerer et førsteordens IIR-filter.
    * Transferfunksjon: $H(z) = \frac{z}{z-\hat{p}}$
    * Impulsrespons: $h[n] = u[n] \cdot \hat{p}\ ^n$
<img src="Figurer/16_DigitalKontroller/Fig3_Zpole.png" style="width: 80%" />

# Nullpunkt
* Et nullpunkt $z=z_k$ i Z-planet representerer et førsteordens FIR filter.
    * Transferfunksjon $H(z) = 1-z_k\cdot z^{-1}$ 
    * Impulsrespons $h[n] = \delta[n] - z_k\cdot \delta[n-1]$.
    * Ikke realiserbar med fysisk system!

* Ingen åpenbar relasjon mellom nullpunkt i s-planet og nullpunkt i z-planet.

# Impulsinvariantmetoden

* Vi kan diskretisere et tidskontinuerlig system med impulsrespons $h(t)$ ved å finne et digitalt filter der impulsresponsen er $h[n] = h(n\cdot T_s)$.


<img src="Figurer/16_DigitalKontroller/Fig4_ImpulseInvariant.png" style="width: 70%; margin-left: 200px" />

* Antagelsen er at lik impulsrespons betyr like filteregenskaper, ihvertfall for frekvenser $f \ll \frac{f_s}{2}$.
* Vi må for øvrig skalere med $T_s$ for at diskrét konvolusjon skal gi samme resultat som tidskontinuerlig konvolusjon.
$$ h[n] = T_s \cdot h(n\cdot T_s)$$

# Førsteordenssystem

* Sampling av impulsresponsen $h(t) = e^{p\cdot t}\cdot u(t)$:
\begin{align}
h[n] &= T_s\cdot h(n\cdot T_s) \\
&= T_s \cdot e^{p\cdot n \cdot T_s} \cdot u[n] \\
&= T_s \cdot \left(e^{p\cdot T_s} \right) ^n \cdot u[n]
\end{align}
* Utregning av transferfunksjon $H(z) =  \textbf{Z}(h[n])$:
\begin{align}
H(z) &= \textbf{Z}(h[n])\\
&=  \textbf{Z}\left(T_s \cdot \left(e^{p\cdot T_s} \right)^n \cdot u[n]\right)\\
&= T_s\cdot \frac{z}{z-e^{p\cdot T_s}}
\end{align}
* Avbildning av poler fra s-planet til z-planet:
$$\hat{p} = e^{p \cdot T_s}$$

# Generelt prinsipp
* Vi antar at $H(s)$ med bruk av delbrøkoppspalting kan skrives som en sum av førsteordenssystem:
$$H(s) = \sum_{k=1}^N \frac{c_k}{s-p_k}$$
* Da vil vi finne et tidsdiskrét system der $h[n] = T_s\cdot h(n\cdot T_s)$ ved å summere sammen et sett med førsteordens IIR-filter med poler i $z= e^{p_k\cdot T_s}$:
$$H(z) = T_s \cdot \sum_{k=1}^N \frac{c_k}{z-e^{p_k\cdot T_s}}$$

## Regneeksempel:
* Bruk impulsinvariantmetoden til å designe et digitalt filter for transferfunksjonen $H(s) = \frac{s+0.5}{s^2+s+9.25}$. Maskinvaren som det digitale filteret skal implementeres på opererer med samplingsfrekvens $f_s = 100$ sampler/sekund.

## Kodeeksempel
* Sammenlign bode-plot fra $H(s)$ og $H(z)$ i forrige eksempel.

In [3]:
# Koeffisienter H(s)
b_t = [1, 0.5]
a_t = [1, 1, 9.25]

# Sampleperiode
T_s = 0.01

# Koeffisienter H(z)
b_n = [T_s, 
       -cos(3*T_s)*exp(-0.5*T_s)*T_s]
a_n = [1, 
       -2*exp(-0.5*T_s)*cos(3*T_s), 
       exp(-T_s)]


# Regn ut arrays til bode-diagram
sys_cont = sig.TransferFunction(b_t, a_t)
w1, mag1, phase1 = sys_cont.bode(w = 2*pi*np.logspace(-2, np.log10(1/T_s/2)+1, 501))

sys_disc = sig.TransferFunction(b_n, a_n, dt=T_s)
w2, mag2, phase2 = sys_disc.bode(w = 2*pi*np.logspace(-2, np.log10(1/T_s/2), 501)*T_s)

# Lag figur
plt.close(1); plt.figure(1)

plt.subplot(2,1,1)
plt.grid(True)
plt.semilogx(w1/2/pi, mag1, label=r'$|H(s)|$')
plt.semilogx(w2/2/pi, mag2, label=r'$|H(z)|$')
plt.ylabel(r'Magnitude (dB)')
plt.legend()
plt.plot([1/T_s/2, 1/T_s/2], [0, 0], 'C3:')[0].set_ydata([-1000, 1000])
plt.subplot(2,1,2)
plt.semilogx(w1/2/pi, phase1, label=r'$\angle H(s)$')
plt.semilogx(w2/2/pi, phase2, label=r'$\angle H(z)$')
plt.ylabel(r'Phase (degrees)')
plt.xlabel(r'Frequency (Hz)')

plt.legend()
plt.plot([1/T_s/2, 1/T_s/2], [0, 0], 'C3:')[0].set_ydata([-1000, 1000])
plt.grid(True)

# Skriv ut digitale filterkoeffisienter
print('b:', b_n)
print('a:', a_n)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

b: [0.01, -0.009945647571577095]
a: [1, -1.9891295143154188, 0.9900498337491681]


# Filterimplementasjon
#### Premiss:
* Vi tar utgangspunkt i at vi har identifisert alle de ønskede filterkoeffisientene $b_k$ og $a_k$.
    * Vi bruker koeffisientverdiene $b_k$ og $a_k$ fra forrige regneeksempel.

#### Utfordring:
* For å implementere filteret må vi kunne skrive et program som utfører beregningene i filteralgoritmen.
    * Det er ønskelig å unngå unødvendig bruk av systemressurser.

## Filtre i Arduino C

* Begynner med globale variabler for filterorden $N$, samt koeffisienter $b_k$ og $a_k$:

<pre>
<font color="#00979c">const</font> <font color="#00979c">int</font> <font color="#000000">N</font> <font color="#434f54">=</font> <font color="#000000">2</font><font color="#000000">;</font> <font color="#434f54">&#47;&#47; Filter order</font>
<font color="#00979c">const</font> <font color="#00979c">float</font> <font color="#000000">b</font><font color="#000000">[</font><font color="#000000">N</font><font color="#434f54">+</font><font color="#000000">1</font><font color="#000000">]</font> <font color="#434f54">=</font> <font color="#000000">{</font><font color="#000000">0.01</font><font color="#434f54">,</font> <font color="#434f54">-</font><font color="#000000">0.009945647571577095</font><font color="#434f54">,</font> <font color="#000000">0.0</font><font color="#000000">}</font><font color="#000000">;</font> <font color="#434f54">&#47;&#47; Coefficients b</font>
<font color="#00979c">const</font> <font color="#00979c">float</font> <font color="#000000">a</font><font color="#000000">[</font><font color="#000000">N</font><font color="#434f54">+</font><font color="#000000">1</font><font color="#000000">]</font> <font color="#434f54">=</font> <font color="#000000">{</font><font color="#000000">1.0</font><font color="#434f54">,</font> <font color="#434f54">-</font><font color="#000000">1.9891295143154188</font><font color="#434f54">,</font> <font color="#000000">0.9900498337491681</font><font color="#000000">}</font><font color="#000000">;</font> <font color="#434f54">&#47;&#47; Coefficients a</font>
</pre>

* Neste steg er å lage en funkjson som kan ta én sample av inngangssignalet $x[n]$ som inngangsparameter, og regne ut korresponderende utgangsverdi $y[n]$.

# Blokkskjema

* Grafisk representasjon av ulike typer filteralgoritmer.
    * Vi antar i alle eksempler at filteret er normalisert slik at $a_0 = 1$
* Noen forskjellige filteralgoritmer / filterstrukturer:
    * Direkte Form 1 (DF1)
    * Direkte Form 2 (DF2)
    * Transponert Direkte Form (TDF)
    

# Direkte Form 1
<img src="Figurer/16_DigitalKontroller/Fig5_DF1.png" style="width: 60%; margin-left: 200px" />

* Alt utregnes i ett steg:
$$y[n] = \sum_{k=0}^{M} b_k \cdot x[n-k] - \sum_{k=1}^{N} a_k \cdot y[n-k]$$
* Krever $M + N$ antall sampler lagret i "buffer".

## Arduino C implementasjon

<pre>
<font color="#00979c">float</font> <font color="#000000">xnk</font><font color="#000000">[</font><font color="#000000">N</font><font color="#000000">]</font> <font color="#000000">{</font><font color="#000000">}</font><font color="#000000">;</font>
<font color="#00979c">float</font> <font color="#000000">ynk</font><font color="#000000">[</font><font color="#000000">N</font><font color="#000000">]</font> <font color="#000000">{</font><font color="#000000">}</font><font color="#000000">;</font>

<font color="#00979c">float</font> <font color="#000000">filter_DF1</font><font color="#000000">(</font><font color="#00979c">float</font> <font color="#000000">x</font><font color="#000000">)</font>
<font color="#000000">{</font>
 &nbsp;<font color="#95a5a6">&#47;* Calculate output using difference equation directly:</font>
<font color="#95a5a6"> &nbsp;&nbsp;* y[n] = b0*x[n] + b1*x[n_1] - a1*y[n-1] + ...</font>
<font color="#95a5a6"> &nbsp;&nbsp;*&#47;</font>
 &nbsp;<font color="#00979c">float</font> <font color="#000000">y</font> <font color="#434f54">=</font> <font color="#000000">b</font><font color="#000000">[</font><font color="#000000">0</font><font color="#000000">]</font><font color="#434f54">*</font><font color="#000000">x</font><font color="#000000">;</font>
 &nbsp;<font color="#5e6d03">for</font><font color="#000000">(</font><font color="#00979c">int</font> <font color="#000000">i</font><font color="#434f54">=</font><font color="#000000">0</font><font color="#000000">;</font> <font color="#000000">i</font><font color="#434f54">&lt;</font><font color="#000000">N</font><font color="#000000">;</font> <font color="#000000">i</font><font color="#434f54">++</font><font color="#000000">)</font>
 &nbsp;<font color="#000000">{</font>
 &nbsp;&nbsp;&nbsp;<font color="#000000">y</font> <font color="#434f54">+=</font> <font color="#000000">b</font><font color="#000000">[</font><font color="#000000">i</font><font color="#434f54">+</font><font color="#000000">1</font><font color="#000000">]</font><font color="#434f54">*</font><font color="#000000">xnk</font><font color="#000000">[</font><font color="#000000">i</font><font color="#000000">]</font> <font color="#434f54">-</font> <font color="#000000">a</font><font color="#000000">[</font><font color="#000000">i</font><font color="#434f54">+</font><font color="#000000">1</font><font color="#000000">]</font><font color="#434f54">*</font><font color="#000000">ynk</font><font color="#000000">[</font><font color="#000000">i</font><font color="#000000">]</font><font color="#000000">;</font>
 &nbsp;<font color="#000000">}</font>
 &nbsp;<font color="#95a5a6">&#47;* &#34;Roll&#34; the buffers containing x[n-k] and y[n-k] one</font>
<font color="#95a5a6"> &nbsp;&nbsp;* step. This can and should be avoided using a sample </font>
<font color="#95a5a6"> &nbsp;&nbsp;* counter variable.</font>
<font color="#95a5a6"> &nbsp;&nbsp;*&#47;</font>
 &nbsp;<font color="#5e6d03">for</font><font color="#000000">(</font><font color="#00979c">int</font> <font color="#000000">i</font><font color="#434f54">=</font><font color="#000000">N</font><font color="#434f54">-</font><font color="#000000">1</font><font color="#000000">;</font> <font color="#000000">i</font><font color="#434f54">&gt;</font><font color="#000000">0</font><font color="#000000">;</font> <font color="#000000">i</font><font color="#434f54">--</font><font color="#000000">)</font>
 &nbsp;<font color="#000000">{</font>
 &nbsp;&nbsp;&nbsp;<font color="#000000">xnk</font><font color="#000000">[</font><font color="#000000">i</font><font color="#000000">]</font> <font color="#434f54">=</font> <font color="#000000">xnk</font><font color="#000000">[</font><font color="#000000">i</font><font color="#434f54">-</font><font color="#000000">1</font><font color="#000000">]</font><font color="#000000">;</font>
 &nbsp;&nbsp;&nbsp;<font color="#000000">ynk</font><font color="#000000">[</font><font color="#000000">i</font><font color="#000000">]</font> <font color="#434f54">=</font> <font color="#000000">ynk</font><font color="#000000">[</font><font color="#000000">i</font><font color="#434f54">-</font><font color="#000000">1</font><font color="#000000">]</font><font color="#000000">;</font>
 &nbsp;<font color="#000000">}</font>
 &nbsp;<font color="#5e6d03">return</font> <font color="#000000">y</font><font color="#000000">;</font>
<font color="#000000">}</font>
</pre>

# Direkte Form 2
<img src="Figurer/16_DigitalKontroller/Fig6_DF2_int.png" style="width: 60%; margin-left: 200px" />

* Deler opp filteret fra DF1 i én rekursiv, og én ikke-rekursiv del.
* Ettersom dette er LTI-system, kan man enkelt bytte om på rekkefølgen.
* $y_1[n]$ og $x_2[n]$ er samme signal.


# Direkte Form 2
<img src="Figurer/16_DigitalKontroller/Fig7_DF2.png" style="width: 50%; margin-left: 200px" />

* Utregnes i to steg:
\begin{align}
w[n] &= x[n] - \sum_{k=1}^{N} a_k \cdot y[n-k]\\
y[n] &= \sum_{k=0}^{M} b_k \cdot w[n-k]
\end{align}
* Krever kun $\text{max}(M, N)$ antall sampler lagret i "buffer".

## Arduino C implementasjon
<pre>
<font color="#00979c">float</font> <font color="#000000">wnk</font><font color="#000000">[</font><font color="#000000">N</font><font color="#000000">]</font> <font color="#000000">{</font><font color="#000000">}</font><font color="#000000">;</font>

<font color="#00979c">float</font> <font color="#000000">filter_DF2</font><font color="#000000">(</font><font color="#00979c">float</font> <font color="#000000">x</font><font color="#000000">)</font>
<font color="#000000">{</font>
 &nbsp;<font color="#95a5a6">&#47;* Calculate intermediary output in first section</font>
<font color="#95a5a6"> &nbsp;&nbsp;* w[n] = x[n] -a1*w[n-1] + a2*x[n-2] +...</font>
<font color="#95a5a6"> &nbsp;&nbsp;*&#47;</font>
 &nbsp;<font color="#00979c">float</font> <font color="#000000">w</font> <font color="#434f54">=</font> <font color="#000000">x</font><font color="#000000">;</font>
 &nbsp;<font color="#5e6d03">for</font><font color="#000000">(</font><font color="#00979c">int</font> <font color="#000000">i</font><font color="#434f54">=</font><font color="#000000">0</font><font color="#000000">;</font> <font color="#000000">i</font><font color="#434f54">&lt;</font><font color="#000000">N</font><font color="#000000">;</font> <font color="#000000">i</font><font color="#434f54">++</font><font color="#000000">)</font>
 &nbsp;<font color="#000000">{</font>
 &nbsp;&nbsp;&nbsp;<font color="#000000">w</font> <font color="#434f54">-=</font> <font color="#000000">a</font><font color="#000000">[</font><font color="#000000">i</font><font color="#434f54">+</font><font color="#000000">1</font><font color="#000000">]</font><font color="#434f54">*</font><font color="#000000">wnk</font><font color="#000000">[</font><font color="#000000">i</font><font color="#000000">]</font><font color="#000000">;</font>
 &nbsp;<font color="#000000">}</font>
 &nbsp;<font color="#95a5a6">&#47;* Calculate final output in second section</font>
<font color="#95a5a6"> &nbsp;&nbsp;* y[n] = b0*w[n] + b1*w[n-1] + b2*w[n-2] +...</font>
<font color="#95a5a6"> &nbsp;&nbsp;*&#47;</font>
 &nbsp;<font color="#00979c">float</font> <font color="#000000">y</font> <font color="#434f54">=</font> <font color="#000000">w</font><font color="#434f54">*</font><font color="#000000">b</font><font color="#000000">[</font><font color="#000000">0</font><font color="#000000">]</font><font color="#000000">;</font>
 &nbsp;<font color="#5e6d03">for</font><font color="#000000">(</font><font color="#00979c">int</font> <font color="#000000">i</font><font color="#434f54">=</font><font color="#000000">0</font><font color="#000000">;</font> <font color="#000000">i</font><font color="#434f54">&lt;</font><font color="#000000">N</font><font color="#000000">;</font> <font color="#000000">i</font><font color="#434f54">++</font><font color="#000000">)</font>
 &nbsp;<font color="#000000">{</font>
 &nbsp;&nbsp;&nbsp;<font color="#000000">y</font> <font color="#434f54">+=</font> <font color="#000000">b</font><font color="#000000">[</font><font color="#000000">i</font><font color="#434f54">+</font><font color="#000000">1</font><font color="#000000">]</font><font color="#434f54">*</font><font color="#000000">wnk</font><font color="#000000">[</font><font color="#000000">i</font><font color="#000000">]</font><font color="#000000">;</font>
 &nbsp;<font color="#000000">}</font>
 &nbsp;<font color="#95a5a6">&#47;* &#34;Roll&#34; the buffer containing w[n-k] one</font>
<font color="#95a5a6"> &nbsp;&nbsp;* step. This can and should be avoided using a sample </font>
<font color="#95a5a6"> &nbsp;&nbsp;* counter variable.</font>
<font color="#95a5a6"> &nbsp;&nbsp;*&#47;</font>
 &nbsp;<font color="#5e6d03">for</font><font color="#000000">(</font><font color="#00979c">int</font> <font color="#000000">i</font><font color="#434f54">=</font><font color="#000000">N</font><font color="#434f54">-</font><font color="#000000">1</font><font color="#000000">;</font> <font color="#000000">i</font><font color="#434f54">&gt;</font><font color="#000000">0</font><font color="#000000">;</font> <font color="#000000">i</font><font color="#434f54">--</font><font color="#000000">)</font>
 &nbsp;<font color="#000000">{</font>
 &nbsp;&nbsp;&nbsp;<font color="#000000">wnk</font><font color="#000000">[</font><font color="#000000">i</font><font color="#000000">]</font> <font color="#434f54">=</font> <font color="#000000">wnk</font><font color="#000000">[</font><font color="#000000">i</font><font color="#434f54">-</font><font color="#000000">1</font><font color="#000000">]</font><font color="#000000">;</font>
 &nbsp;<font color="#000000">}</font>
 &nbsp;<font color="#5e6d03">return</font> <font color="#000000">y</font><font color="#000000">;</font>
<font color="#000000">}</font>
</pre>

# Transponert Direkte Form
<img src="Figurer/16_DigitalKontroller/Fig8_TDF.png" style="width: 50%; margin-left: 200px" />

* Ligner på Direkte Form 1, men tidsforsinkelsen skjer etter addisjon.
    * "halvferdige" utgangsverdier ligger i buffer.
* Krever kun $\text{max}(M, N)$ antall sampler lagret i buffer.


## Transponert direkte form utledning:
\begin{align}
y[n] &= b_0\cdot x[n] + v_1[n-1]\\
 &= b_0\cdot x[n] + b_1\cdot x[n-1] - a_1 \cdot y[n-1] + v_2[n-2]\\
 &= b_0\cdot x[n] + b_1\cdot x[n-1] - a_1 \cdot y[n-1] + b_2\cdot x[n-2] - a_2 \cdot y[n-2] + v_3[n-3]\\
 &\vdots
\end{align}

## Arduino C implementasjon
<pre>
<font color="#00979c">float</font> <font color="#000000">vk</font><font color="#000000">[</font><font color="#000000">N</font><font color="#000000">]</font> <font color="#000000">{</font><font color="#000000">}</font><font color="#000000">;</font>

<font color="#00979c">float</font> <font color="#000000">filter_TDF</font><font color="#000000">(</font><font color="#00979c">float</font> <font color="#000000">x</font><font color="#000000">)</font>
<font color="#000000">{</font>
 &nbsp;<font color="#95a5a6">&#47;* Calculate Output y[n]: *&#47;</font>
 &nbsp;<font color="#00979c">float</font> <font color="#000000">y</font> <font color="#434f54">=</font> <font color="#000000">b</font><font color="#000000">[</font><font color="#000000">0</font><font color="#000000">]</font><font color="#434f54">*</font><font color="#000000">x</font> <font color="#434f54">+</font> <font color="#000000">vk</font><font color="#000000">[</font><font color="#000000">0</font><font color="#000000">]</font><font color="#000000">;</font>
 &nbsp;<font color="#95a5a6">&#47;* Update partial output signals v_k[n]:</font>
<font color="#95a5a6"> &nbsp;&nbsp;*&#47;</font>
 &nbsp;<font color="#5e6d03">for</font><font color="#000000">(</font><font color="#00979c">int</font> <font color="#000000">i</font><font color="#434f54">=</font><font color="#000000">1</font><font color="#000000">;</font> <font color="#000000">i</font><font color="#434f54">&lt;</font><font color="#000000">N</font><font color="#000000">;</font> <font color="#000000">i</font><font color="#434f54">++</font><font color="#000000">)</font>
 &nbsp;<font color="#000000">{</font>
 &nbsp;&nbsp;&nbsp;<font color="#000000">vk</font><font color="#000000">[</font><font color="#000000">i</font> <font color="#434f54">-</font> <font color="#000000">1</font><font color="#000000">]</font> <font color="#434f54">=</font> <font color="#000000">x</font> <font color="#434f54">*</font> <font color="#000000">b</font><font color="#000000">[</font><font color="#000000">i</font><font color="#000000">]</font> <font color="#434f54">-</font> <font color="#000000">y</font> <font color="#434f54">*</font> <font color="#000000">a</font><font color="#000000">[</font><font color="#000000">i</font><font color="#000000">]</font> <font color="#434f54">+</font> <font color="#000000">vk</font><font color="#000000">[</font><font color="#000000">i</font><font color="#000000">]</font><font color="#000000">;</font>
 &nbsp;<font color="#000000">}</font>
 &nbsp;<font color="#000000">vk</font><font color="#000000">[</font><font color="#000000">N</font><font color="#434f54">-</font><font color="#000000">1</font><font color="#000000">]</font> <font color="#434f54">=</font> <font color="#000000">x</font> <font color="#434f54">*</font> <font color="#000000">b</font><font color="#000000">[</font><font color="#000000">N</font><font color="#000000">]</font> <font color="#434f54">-</font> <font color="#000000">y</font> <font color="#434f54">*</font> <font color="#000000">a</font><font color="#000000">[</font><font color="#000000">N</font><font color="#000000">]</font><font color="#000000">;</font>
 &nbsp;<font color="#95a5a6">&#47;* Return output sample value: *&#47;</font>
 &nbsp;<font color="#5e6d03">return</font> <font color="#000000">y</font><font color="#000000">;</font>
<font color="#000000">}</font>
</pre>

# Oppsummering Digitale Filtre

* LTI-systemer og Differanseligningen
* Impulsrespons og Konvolusjon
* Frekvensrespons
* Z-transformasjon, poler og nullpunkt